In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
from main import executor
from fonte_dados.fabrica import FabricaFonteDados
from similarity.similarity import SimilarityCalculator
from treinamento import treinamento_lda
from treinamento.treinamento_lda import TreinamentoLda
from util import constants

import numpy as np
import pandas as pd
import time

In [ ]:
fabrica = FabricaFonteDados()

In [ ]:
origem = constants.NERDS_VIAJANTES
destino = constants.WIKIPEDIA

## Executar treinamento com código reusável

In [ ]:
resultado_df = executor.executar_treinamento(fonte_origem=origem, fonte_destino=destino, num_topics=100)

## Trabalho com base de treinamento

In [ ]:
fonte_dados_origem = fabrica.get_fonte_dados(origem)
fonte_dados_origem.carregar_dados()
documentos_origem = fonte_dados_origem.get_tokens()

In [ ]:
fonte_dados_origem.get_dataframe().head()

### Ajuste de modelo para posts de base de treinamento (origem)

In [ ]:
# Ajusta modelo LDA para os documentos de origem. O resultado gerado contem o dicionario, o corpus de dados e o modelo gerado
treinamento_lda = TreinamentoLda(num_topics=100, passes=2)
resultado_lda = treinamento_lda.ajustar_modelo(documentos_origem)

In [ ]:
resultado_lda

### Definicao de probabilidades de topicos para documentos de origem

In [ ]:
probabilidades_topicos_origem = [treinamento_lda.calcular_probabilidades_documento(dnv, resultado_lda) for dnv in documentos_origem]
np.array(probabilidades_topicos_origem).shape

### Teste com classe

## Trabalho com base de comparação (destino)

In [ ]:
fonte_dados_destino = fabrica.get_fonte_dados(destino)
fonte_dados_destino.carregar_dados()
documentos_destino = fonte_dados_destino.get_tokens()

In [ ]:
fonte_dados_destino.get_dataframe().head()

### Definicao de topicos para documentos de destino

- Quantidade de documentos: 2252
- Indice do maior documento: 123
- Tamanho do maior documento: 10018 tokens
- Tempo gasto para definir tópicos de maior documento: 0.02167201042175293 segundos
- Tempo estimado para todos os documentos (com base no tempo do maior): 18.83 segundos
    * Na verdade este valor variou muito, não é fixo, mas a maioria das vezes ficou em torno de 20 segundos

In [ ]:
# TODO: Remover
def avaliar_tempos_definicao_topicos_wikipedia():
    print(f'Quantidade de documentos: {len(documentos_wikipedia)}')

    tamanhos = [len(d) for d in documentos_wikipedia]
    indice_maior_documento = np.argmax(tamanhos)
    print(f'Indice do maior documento: {indice_maior_documento}')
    print(f'Tamanho do maior documento: {tamanhos[indice_maior_documento]} tokens')

    maior_documento = documentos_wikipedia[indice_maior_documento]
    start_time = time.time()
    treinamento_lda.calcular_probabilidades_documento(maior_documento, lda_nerds_viajantes)
    tempo_gasto_maior_documento = time.time() - start_time
    print(f"Tempo gasto na definicao dos topicos do maior documento: {tempo_gasto_maior_documento:.2f} segundos")
    print(f'Tempo estimado para todos documentos: {len(documentos_wikipedia) * tempo_gasto_maior_documento:.2f} segundos')

# avaliar_tempos_definicao_topicos_wikipedia()

### Calculo de probabilidade de topicos para documentos de destino

In [ ]:
start_time = time.time()
probabilidades_topicos_destino = [
    treinamento_lda.calcular_probabilidades_documento(dw, resultado_lda) for dw in documentos_destino]
elapsed_time = time.time() - start_time
print(f'Tempo total gasto na definicao dos topicos: {elapsed_time:.2f} segundos')

## Calculo de semelhanca entre documentos de treinamento e comparação

In [ ]:
print(f'Quantidade de documentos de origem: {len(probabilidades_topicos_origem)}')
print(f'Tamanho do array de probabilidades de documentos de origem: {len(probabilidades_topicos_origem[0])}')
print(f'Quantidade de documentos de destino: {len(probabilidades_topicos_destino)}')
print(f'Tamanho do array de probabilidades de documentos de destino: {len(probabilidades_topicos_destino[0])}')

### Problema encontrado

Ao calcular as probabilidades que determinados documentos estejam em determinados tópicos vimos que poucos tópicos determinam um documento, independente de usarmos 13 ou 100 tópicos no modelo. Para calcular a matrix esparsa das similaridades eu acabei tendo que preencher um array com 0 em quase todas as posições para que não desse erro.
Devemos estudar para ver se o modelo LDA é realmente a melhor abordagem neste caso ou se não temos muitos poucos documentos.

Sem preencher com 0's os tópicos que não contribuiam para cada documento o que acontecia era que cada array de probabilidade por tópico de um documento tinha um tamanho diferente, o que dava erro ao criar a matrix esparsa, que espera que todos tenham a mesma quantidade de colunas, que no caso é a quantidade de tópicos.

UPDATE: Este documento não mais representa um problema pois ao usar o método calcular_probabilidades_documento este já retorno exatamente a quantidade de tópicos utilizada no treinamento.

In [ ]:
similarity_calculator = SimilarityCalculator(probabilidades_topicos_destino)

### Calcula as documentos de comparação mais parecidos com os documentos de treinamento

In [ ]:
destinos_mais_parecidos = [similarity_calculator.get_most_similar_documents(pto) for pto in probabilidades_topicos_origem]
destino_mais_parecido = [dmp[0] for dmp in destinos_mais_parecidos]

In [ ]:
print(destinos_mais_parecidos[:5])
print(destino_mais_parecido[:5])

## Montagem de resultado

In [ ]:
destino_df = fonte_dados_destino.get_dataframe()

titulos_destino = destino_df['titulo'].values
titulos_mais_parecidos = [titulos_destino[dmp] for dmp in destino_mais_parecido]

ids_documentos_destino = destino_df['id_documento'].values
ids_documentos_mais_parecidos = [ids_documentos_destino[dmp] for dmp in destino_mais_parecido]

print(titulos_mais_parecidos[:10])
print(len(titulos_mais_parecidos))

print(ids_documentos_mais_parecidos[:10])
print(len(ids_documentos_mais_parecidos))

In [ ]:
resultado_df = pd.DataFrame(data = {
    'id_documento_origem': fonte_dados_origem.get_dataframe()['id_documento'].values,
    'titulo_documento_origem': fonte_dados_origem.get_dataframe()['titulo'].values,
    'id_documento_destino': ids_documentos_mais_parecidos,
    'titulo_documento_destino': titulos_mais_parecidos
})
resultado_df.head()

## Análise específica - Nervs Viajantes versus Wikipedia

### Uma análise de post específico (bariloche-cerro-tronador-e-cascada-los-alerces)

Post escolhido do Nerds viajantes para analisar:

- id: 1356
- indice no array: 49
- name: bariloche-cerro-tronador-e-cascada-los-alerces

In [ ]:
indice_post_nv = 49
posts.iloc[49][['name', 'title']]

In [ ]:
bow_49 = lda_nerds_viajantes.corpus[49]
topics = lda_nerds_viajantes.modelo_lda.get_document_topics(bow_49)

"""
Topicos identificados para documento 49: 
[(6, 0.23247316), (91, 0.7655891)]
"""
# topics

# Visualizando topico 91, que contribui em 77%
lda_nerds_viajantes.modelo_lda.show_topic(topicid=91, topn=20)

In [ ]:
from collections import Counter
print(f'Tamanho do documento 49: {len(documentos[49])}')
print(Counter(documentos[49]))

In [ ]:
paginas_wikipedia_mais_parecidas = wikipedia_mais_parecidos[indice_post_nv]
print(f'Wikipedia mais parecido: {pages_df.iloc[paginas_wikipedia_mais_parecidas, [1, 2]]}')

In [ ]:
print(f'Tamanho do documento Rio Awetere (1119): {len(documentos_wikipedia[1119])}')
corpus_rio_awetere = lda_nerds_viajantes.dicionario.doc2bow(documentos_wikipedia[1119])
topics = lda_nerds_viajantes.modelo_lda.get_document_topics(corpus_rio_awetere)
"""
Topicos que mais contribuem para documento 1119:
Veja que o 91 eh o que mais contribui, dai a semelhanca com o post da cascada los alerces
[(6, 0.21784398), (53, 0.03848416), (87, 0.09624962), (91, 0.61537075)]
"""
print(topics)

### Trabalho futuro

- Considerar remocao de palavras que estao contribuindo muito para formacao de determinados topicos e poderiam ser inuteis.
    * A palavra "dia", por exemplo, está aparecendo muito e parece pouco contribuir para a semelhança entre os documentos
- Considerar a possibilidade de traduzir algumas palavras que aparecem muito, como cascada, por exemplo

**Comentários sobre o resultado e trabalho futuro**

- Todos os posts que vem como os mais parecidos são relacionados a rios
- Os rios dos resultados são distribuídos entre os países Estados Unidos e Nova Zelândia mas curiosamente não Argentina
    * Provavelmente eu não baixei os posts de rios da Argentina

**Trabalho futuro**

- Analisar qual tópico mais aparece para este post
    * Analisar também do mais parecido na wikipedia
- Analisar palavras que mais contribuem para os tópicos em questão

### Análise com post específico

- circuito-w-las-torres-del-paine
    * id: 8218, indice: 256
- el-chalten-trilha-para-laguna-torre
    * id: 9829, indice: 290

In [ ]:
posts.reset_index(inplace=True)
posts[posts['name'].str.contains('torre')]

In [ ]:
indice_post_nv = 257
posts.iloc[indice_post_nv][['name', 'title']]

In [ ]:
paginas_wikipedia_mais_parecidas = wikipedia_mais_parecidos[indice_post_nv]
print(f'Wikipedia mais parecido: {pages_df.iloc[paginas_wikipedia_mais_parecidas, [1, 2]]}')

### Análise de tópicos

In [ ]:
documento = documentos.values[indice_post_nv]
bow = [lda_nerds_viajantes.dicionario.doc2bow(documento)]

In [ ]:
topics = lda_nerds_viajantes.modelo_lda.get_document_topics(bow)

In [ ]:
"""
Resultado: Ha praticamente 100% de chance de o documento ser definido pelo topico 20
"""
for topic in topics:
    print(topic)

In [ ]:
lda_nerds_viajantes.modelo_lda.print_topic(20)